## Intall Dependencies

In [1]:
%%capture
!pip install unsloth unsloth_zoo python-dotenv

## Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup Model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel
from unsloth import add_new_tokens
from peft import PeftModel
import torch

# ===================
# = LOAD BASE MODEL =
# ===================

max_seq_length = 2048
dtype = None
load_in_4bit = True

base_model, _ = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# ============================
# = ADD TOKENS TO BASE MODEL =
# ============================

# Define special tokens
special_tokens = {
    '<SEARCH>',
    '<\SEARCH>',
    '<RESULT>',
    '<\RESULT>'
}

# Exclude existing tokens
new_tokens = list(special_tokens - set(_.vocab.keys()))

if new_tokens:
    print("Addding new tokens... ")

    # Add new tokens to tokenizer and model using add_new_tokens()
    add_new_tokens(base_model, _, new_tokens=new_tokens)

    # Verify the additions
    print(f"Added {len(new_tokens)} new tokens to the tokenizer and model! ")
else:
    print("No new tokens added. ")


# =============================
# = ADD ADAPTER TO BASE MODEL =
# =============================

model = PeftModel.from_pretrained(base_model, "canertugrul/DeepSeek-R1-Distill-Llama-8B-Tool-Use-Adapter")

FastLanguageModel.for_inference(model)


# ==================
# = LOAD TOKENIZER =
# ==================

tokenizer = AutoTokenizer.from_pretrained("canertugrul/DeepSeek-R1-Distill-Llama-8B-Tool-Use-Tokenizer")

<ipython-input-4-1ca3c3ed351d>:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/858 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.47G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Perform Inference

In [5]:
inference_system_prompt = """Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a scientific expert with advanced knowledge in analytical reasoning, problem-solving, and quantitative analysis across various disciplines, including mathematics, physics, biology, chemistry, and engineering. Please answer the following question.

### Question:
{}

### Response:
<think>
{}
"""

question = "Who is the current president of the United States? What did they mention in their latest speech?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([inference_system_prompt.format(question, "")], return_tensors="pt").to(model.device)

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

print(response[0])

<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a scientific expert with advanced knowledge in analytical reasoning, problem-solving, and quantitative analysis across various disciplines, including mathematics, physics, biology, chemistry, and engineering. Please answer the following question.

### Question:
Who is the current president of the United States? What did they mention in their latest speech?

### Response:
<think>

To determine the current president of the United States and their latest speech, I will first retrieve the latest information on the U.S. presidency. 

<SEARCH> Current president of the United States </SEARCH>

<RESULT> The current president of the United States is Joe 